In [ ]:
#! pip3 install tqdm

In [ ]:
import requests
import time
from tqdm import tqdm  # 진행률 표시 라이브러리

# --- 1. 설정값 ---

# API 기본 URL
BASE_URL = "https://api.musinsa.com/api2/dp/v1/plp/goods"
# 고정 파라미터
# page는 루프에서 자동으로 추가됩니다.
PARAMS = {
    'gf': 'M',
    'sortCode': 'SALE_ONE_YEAR_COUNT',
    'category': '103',
    'size': 60,
    'testGroup': '',
    'caller': 'CATEGORY',
    'seen': 0,
    'seenAds': ''
}

# 서버가 크롤러임을 인지하지 못하도록 일반 브라우저처럼 User-Agent 설정
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# 총 페이지 수
TOTAL_PAGES = 1312
# 요청 간 최소 간격 (초) - 서버 보호를 위해 필수!
SLEEP_TIME = 1
# 저장할 파일명
OUTPUT_FILE = "musinsa_product_ids_sorted_by_sales_volume.txt"


# --- 2. 메인 크롤링 함수 ---

def fetch_all_product_ids():
    """1페이지부터 TOTAL_PAGES까지 순회하며 모든 상품 ID를 수집합니다."""
    
    all_product_ids = []
    
    print(f"총 {TOTAL_PAGES} 페이지의 상품 ID 수집을 시작합니다.")
    
    # tqdm을 사용하여 루프의 진행 상태를 표시합니다.
    for page_num in tqdm(range(1, TOTAL_PAGES + 1), desc="페이지 수집 중"):
        
        # 1. 요청 파라미터 설정 (현재 페이지 번호 추가)
        current_params = PARAMS.copy()
        current_params['page'] = page_num
        
        try:
            # 2. API 요청
            response = requests.get(BASE_URL, params=current_params, headers=HEADERS, timeout=10)
            
            # 요청이 성공했는지 확인 (4xx, 5xx 에러 시 예외 발생)
            response.raise_for_status() 
            
            data = response.json()
            
            # [!!!] 핵심 파싱 부분 (사용자 JSON 구조에 맞게 수정됨) [!!!]
            # (수정된 경로: data -> list -> 리스트의 각 요소 -> goodsNo)
            
            if 'data' in data and 'list' in data['data']: # 'goodsList'를 'list'로 변경
                goods_list = data['data']['list']      # 'goodsList'를 'list'로 변경
                
                if not goods_list:
                    # 데이터가 비어있으면 루프 중단 (예: 마지막 페이지)
                    print(f"\nInfo: {page_num} 페이지에 데이터가 없습니다. 중단합니다.")
                    break
                
                # 'goodsNo' 키를 가진 ID만 추출
                for item in goods_list:
                    if 'goodsNo' in item:
                        all_product_ids.append(item['goodsNo'])
            
            else:
                # 예상한 구조가 아닐 경우
                print(f"\nWarning: {page_num} 페이지에서 'data' 또는 'list' 키를 찾지 못했습니다.")
                continue # 다음 페이지로 넘어감

            # 3. 서버 부하를 줄이기 위한 대기 시간 (필수!)
            # 1312번 연속 요청 시 차단될 수 있습니다.
            time.sleep(SLEEP_TIME) 

        except requests.exceptions.HTTPError as http_err:
            print(f"\nHTTP Error on page {page_num}: {http_err}")
            # 403 (Forbidden)이나 429 (Too Many Requests) 에러 시 대기 시간 늘리기
            if response.status_code == 403 or response.status_code == 429:
                print("서버가 요청을 거부했습니다. 10초 대기 후 다시 시도합니다.")
                time.sleep(10)
            
        except requests.exceptions.RequestException as req_err:
            # 네트워크 연결 오류 등
            print(f"\nRequest Error on page {page_num}: {req_err}")
            # 5초 대기 후 다음 페이지 시도
            time.sleep(5)
        
        except Exception as e:
            # JSON 파싱 오류 등 기타 예외
            print(f"\nAn error occurred on page {page_num}: {e}")
            print(f"Response (text): {response.text[:200]}...") # 응답 내용 일부 출력
            

    return all_product_ids


# --- 3. 파일 저장 ---

def save_ids_to_file(product_ids):
    """수집된 ID 리스트를 텍스트 파일로 저장합니다."""
    
    print(f"\n총 {len(product_ids)}개의 ID를 수집했습니다.")
    
    # 중복 제거 (필요시)
    unique_ids = list(set(product_ids))
    if len(unique_ids) < len(product_ids):
        print(f"중복 제거 후 {len(unique_ids)}개의 ID가 남았습니다.")
    else:
        print("중복된 ID가 없습니다.")

    # 파일로 저장 (한 줄에 하나의 ID)
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            for pid in unique_ids:
                f.write(f"{pid}\n")
        
        print(f"'{OUTPUT_FILE}' 파일에 성공적으로 저장했습니다.")
        
    except IOError as e:
        print(f"Error: 파일 저장에 실패했습니다. {e}")


# --- 4. 스크립트 실행 ---

if __name__ == "__main__":
    ids = fetch_all_product_ids()
    
    if ids:
        save_ids_to_file(ids)
    else:
        print("수집된 ID가 없습니다. 프로그램을 종료합니다.")

In [ ]:
import os
# 파일 경로 설정 (현재 스크립트와 같은 디렉토리에 있다고 가정)
FILE_NAME = 'musinsa_product_ids.txt'

def read_product_ids_to_list(file_path):
    """
    지정된 파일 경로에서 상품 ID를 읽어 리스트로 반환합니다.
    각 줄의 공백을 제거하고, 비어있는 줄은 제외합니다.
    """
    if not os.path.exists(file_path):
        print(f"❌ 오류: 지정된 파일 '{file_path}'이(가) 존재하지 않습니다.")
        return []

    product_ids = []
    
    try:
        # 파일을 읽기 모드('r')로 열고, 파일이 자동으로 닫히도록 with 구문 사용
        with open(file_path, 'r', encoding='utf-8') as f:
            # 파일 전체 내용을 읽고, 줄 바꿈 문자('\n')를 기준으로 분리
            lines = f.readlines()
            
            # 각 줄을 순회하며 리스트에 추가 (가장 깔끔하고 효율적인 방법)
            for line in lines:
                # 1. 양쪽의 공백(줄 바꿈 포함) 제거
                stripped_id = line.strip() 
                
                # 2. 비어있는 줄이 아닌 경우에만 리스트에 추가
                if stripped_id:
                    product_ids.append(stripped_id)
        
        print(f"✅ 총 {len(product_ids)}개의 상품 ID를 성공적으로 로드했습니다.")
        return product_ids
        
    except Exception as e:
        print(f"❌ 파일을 읽는 중 예기치 않은 오류가 발생했습니다: {e}")
        return []

# 함수 실행
id_list = read_product_ids_to_list(FILE_NAME)

# 로드된 리스트 확인
if id_list:
    print("\n[로딩된 상품 ID 리스트 확인]")
    print(id_list)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://www.musinsa.com/products/4441592'

# 실제 브라우저처럼 보이게 할 헤더 설정
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'Referer': 'https://www.musinsa.com/'
}


# Session 객체 생성
session = requests.Session()

try:
    # Session 객체를 통해 요청을 보냅니다.
    # 서버가 첫 요청(URL)에 쿠키를 포함하여 응답하면,
    # Session 객체가 이 쿠키를 자동으로 저장합니다.
    response = session.get(URL, headers=HEADERS)
    response.raise_for_status() 
    
    html_content = response.text
    
    # 5. BeautifulSoup으로 파싱 및 추출 (이전과 동일)
    soup = BeautifulSoup(html_content, 'lxml')
    
    print("--- Session을 이용한 데이터 추출 결과 ---")

    # 추출 코드 예시
    brand_element = soup.select_one('.product-information .product_title a')
    brand_name = brand_element.get_text(strip=True) if brand_element else "브랜드 정보를 찾을 수 없음"
    print(f"✅ 브랜드 이름: {brand_name}")

    product_name_element = soup.select_one('.product-information span.product_title')
    product_name = product_name_element.get_text(strip=True) if product_name_element else "상품명을 찾을 수 없음"
    print(f"✅ 상품명: {product_name}")
    
    price_element = soup.select_one('.product-order-information .li_price.sale .txt_price')
    price_text = price_element.get_text(strip=True) if price_element else "가격을 찾을 수 없음"
    print(f"✅ 판매 가격: {price_text}")

except requests.exceptions.RequestException as e:
    print(f"요청 처리 중 오류가 발생했습니다: {e}")

In [ ]:
from curl_cffi import requests
from bs4 import BeautifulSoup

# 'impersonate' 파라미터가 핵심입니다. (chrome110 버전인 척 연기)
url = "https://www.musinsa.com/products/4441592"

try:
    response = requests.get(
        url, 
        impersonate="chrome110",  # 크롬 브라우저의 TLS 지문 사용
        headers={
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
        }
    )
    
    # Cloudflare 통과 여부 확인
    if "<title>Just a moment...</title>" in response.text:
        print("❌ Cloudflare에 막혔습니다.")
    else:
        print(f"🎉 성공! 상태 코드: {response.status_code}")
        
        # BeautifulSoup으로 데이터 추출
        soup = BeautifulSoup(response.text, 'lxml')
        title = soup.select_one('.product_title')
        print(title.get_text(strip=True) if title else "제목 못찾음")

except Exception as e:
    print(f"오류 발생: {e}")

In [1]:
import requests
import time
from tqdm import tqdm  # 진행률 표시 바 (pip install tqdm)

def collect_musinsa_tops_ids(target_count=50000):
    # 1. 설정값 정의
    base_url = "https://api.musinsa.com/api2/dp/v1/plp/goods"
    
    # 헤더 설정 (브라우저인 척 속임)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "https://www.musinsa.com/"
    }

    # 수집된 ID를 저장할 리스트
    collected_ids = []
    
    # 페이지당 60개이므로, 5만개를 채우기 위한 예상 페이지 수 계산
    # 50000 / 60 = 833.33... -> 약 834페이지 이상 필요
    # 넉넉하게 850페이지까지 돌리거나, 5만개가 차면 멈추도록 설정
    max_page_estimate = (target_count // 60) + 5 

    print(f"🎯 목표: 상의 카테고리 상품 ID 약 {target_count}개 수집 시작")
    print(f"📄 예상 요청 페이지 수: 약 {max_page_estimate} 페이지")

    # 2. 반복 요청 시작
    for page in tqdm(range(1, max_page_estimate + 1), desc="수집 중"):
        
        # 목표 수량을 채웠으면 중단
        if len(collected_ids) >= target_count:
            print(f"\n✨ 목표 수량 도달! ({len(collected_ids)}개)")
            break

        # 파라미터 설정 (page만 계속 바뀜)
        params = {
            "gf": "A",                         # 성별 (A: 전체)
            "sortCode": "SALE_ONE_YEAR_COUNT", # 정렬 (1년 판매량 순)
            "category": "001",                 # 카테고리 (상의)
            "size": "60",                      # 페이지당 개수
            "page": page,                      # 현재 페이지
            "caller": "CATEGORY"
        }

        try:
            # 요청 보내기
            response = requests.get(base_url, headers=headers, params=params, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                
                # JSON 구조 파싱: data -> data -> list
                # 응답 구조가 가끔 다를 수 있으므로 안전하게 가져오기
                items = data.get("data", {}).get("list", [])
                
                if not items:
                    print(f"\n⚠️ {page}페이지에 상품이 없습니다. 수집을 종료합니다.")
                    break
                
                # ID 추출 및 저장
                for item in items:
                    if "goodsNo" in item:
                        collected_ids.append(item["goodsNo"])
            
            else:
                print(f"\n❌ 요청 실패 (Status: {response.status_code}) - Page: {page}")
                time.sleep(10) # 에러 발생 시 잠시 대기
                continue

            # 3. 중요: 서버 부하 방지 및 차단 회피를 위한 대기 (0.5~1초)
            time.sleep(0.8)

        except Exception as e:
            print(f"\n❗ 에러 발생: {e}")
            time.sleep(5)

    return collected_ids

# --- 실행 및 파일 저장 ---
if __name__ == "__main__":
    # 1. 수집 실행
    final_ids = collect_musinsa_tops_ids(target_count=50000)
    
    # 2. 중복 제거 (혹시 모를 중복 방지)
    unique_ids = list(set(final_ids))
    print(f"\n📊 최종 수집: {len(final_ids)}개 -> 중복 제거 후: {len(unique_ids)}개")

    # 3. 파일로 저장
    filename = "musinsa_tops_ids_50k.txt"
    with open(filename, "w", encoding="utf-8") as f:
        for goods_id in unique_ids:
            f.write(f"{goods_id}\n")
            
    print(f"💾 '{filename}' 파일에 저장 완료!")

수집 중:   0%|          | 0/838 [00:00<?, ?it/s]

🎯 목표: 상의 카테고리 상품 ID 약 50000개 수집 시작
📄 예상 요청 페이지 수: 약 838 페이지


수집 중: 100%|█████████▉| 834/838 [13:21<00:03,  1.04it/s]


✨ 목표 수량 도달! (50040개)

📊 최종 수집: 50040개 -> 중복 제거 후: 49992개
💾 'musinsa_tops_ids_50k.txt' 파일에 저장 완료!


In [ ]:
import pymysql
import sys

# ----------------------------------------------------------------------
# 1. RDS 접속 정보 설정 (PASSWORD를 실제 암호로 변경해야 합니다)
# ----------------------------------------------------------------------
HOST = 'musinsa-data.c07kuo6ug98z.us-east-1.rds.amazonaws.com'
USER = 'admin'
PASSWORD = 'qkqajrwkrnrnrn9_'  # ⚠️ 실제 암호로 변경하세요!
DATABASE = 'musinsa'
PORT = 3306

def create_musinsa_tables():
    """RDS에 접속하여 musinsa 스키마의 모든 테이블을 생성하는 함수"""
    conn = None # 연결 객체 초기화
    
  

    try:
        # 데이터베이스 연결 시도
        conn = pymysql.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            db=DATABASE,
            port=PORT,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.Cursor
        )
        
        cursor = conn.cursor()
        
        print("=" * 40)
        print("🔗 데이터베이스 연결 성공.")
        
        # 3. 기존 테이블 삭제 (순서대로)
        print("🗑️ 기존 테이블 삭제 시도...")
        cursor.execute()

        conn.commit()
        print("✅ 기존 테이블 삭제 완료 (없으면 무시).")
        
    
        
        # product 테이블 구조 확인
        cursor.execute("DESCRIBE product;")
        print("\n[product 테이블]")
        for col in cursor.fetchall():
            print(f"  - 컬럼: {col[0]}, 타입: {col[1]}")

        # crawl_target_id 테이블 구조 확인
        cursor.execute("DESCRIBE crawl_target_id;")
        print("\n[crawl_target_id 테이블]")
        for col in cursor.fetchall():
            print(f"  - 컬럼: {col[0]}, 타입: {col[1]}")
            
        print("-" * 40)
        
    except pymysql.MySQLError as e:
        print(f"❌ 데이터베이스 작업 오류 발생: {e}")
        if conn:
            conn.rollback()
        sys.exit(1)
        
    finally:
        # 연결 해제
        if conn:
            conn.close()
            print("🔗 데이터베이스 연결 해제 완료.")

if __name__ == "__main__":
    create_musinsa_tables()

🔗 데이터베이스 연결 성공.
🗑️ 기존 테이블 삭제 시도...
✅ 기존 테이블 삭제 완료 (없으면 무시).
🏗️ product 테이블 생성 시작...
🎉 product 테이블 생성 완료!
🏗️ crawl_target_id 테이블 생성 시작...
🎉 crawl_target_id 테이블 생성 완료!

--- 생성된 테이블 구조 확인 ---

[product 테이블]
  - 컬럼: product_id, 타입: int
  - 컬럼: product_name, 타입: varchar(255)
  - 컬럼: brand, 타입: varchar(100)
  - 컬럼: original_price, 타입: int
  - 컬럼: sale_price, 타입: int
  - 컬럼: upper_category, 타입: varchar(50)
  - 컬럼: lower_category, 타입: varchar(50)
  - 컬럼: gender, 타입: int
  - 컬럼: rating, 타입: float
  - 컬럼: wish_count, 타입: int
  - 컬럼: review_count, 타입: int
  - 컬럼: size_info, 타입: json
  - 컬럼: discount_rate, 타입: int
  - 컬럼: fit_season, 타입: json
  - 컬럼: cumulative_sales, 타입: varchar(50)

[crawl_target_id 테이블]
  - 컬럼: product_id, 타입: int
  - 컬럼: is_crawled, 타입: tinyint(1)
  - 컬럼: created_at, 타입: timestamp
----------------------------------------
🔗 데이터베이스 연결 해제 완료.


In [9]:
import os
# 파일 경로 설정 (현재 스크립트와 같은 디렉토리에 있다고 가정)

def read_product_ids_to_list(file_path):
    """
    지정된 파일 경로에서 상품 ID를 읽어 리스트로 반환합니다.
    각 줄의 공백을 제거하고, 비어있는 줄은 제외합니다.
    """
    if not os.path.exists(file_path):
        print(f"❌ 오류: 지정된 파일 '{file_path}'이(가) 존재하지 않습니다.")
        return []

    product_ids = []
    
    try:
        # 파일을 읽기 모드('r')로 열고, 파일이 자동으로 닫히도록 with 구문 사용
        with open(file_path, 'r', encoding='utf-8') as f:
            # 파일 전체 내용을 읽고, 줄 바꿈 문자('\n')를 기준으로 분리
            lines = f.readlines()
            
            # 각 줄을 순회하며 리스트에 추가 (가장 깔끔하고 효율적인 방법)
            for line in lines:
                # 1. 양쪽의 공백(줄 바꿈 포함) 제거
                stripped_id = line.strip() 
                
                # 2. 비어있는 줄이 아닌 경우에만 리스트에 추가
                if stripped_id:
                    product_ids.append(stripped_id)
        
        print(f"✅ 총 {len(product_ids)}개의 상품 ID를 성공적으로 로드했습니다.")
        return product_ids
        
    except Exception as e:
        print(f"❌ 파일을 읽는 중 예기치 않은 오류가 발생했습니다: {e}")
        return []


In [10]:
id_list = read_product_ids_to_list('musinsa_tops_ids_50k.txt')

✅ 총 49992개의 상품 ID를 성공적으로 로드했습니다.


In [13]:
import pymysql
import sys

# ----------------------------------------------------------------------
# 1. RDS 접속 정보 설정 (PASSWORD를 실제 암호로 변경해야 합니다)
# ----------------------------------------------------------------------
HOST = 'musinsa-data.c07kuo6ug98z.us-east-1.rds.amazonaws.com'
USER = 'admin'
PASSWORD = 'qkqajrwkrnrnrn9_'  # ⚠️ 실제 암호로 변경하세요!
DATABASE = 'musinsa'
PORT = 3306

def insert_target_ids(product_id_list):
    """
    주어진 상품 ID 리스트를 crawl_target_id 테이블에 일괄 삽입합니다.
    (대량 삽입 최적화를 위해 executemany 사용)
    """
    conn = None
    if not product_id_list:
        print("⚠️ 삽입할 상품 ID 리스트가 비어 있습니다. 작업을 건너뜁니다.")
        return

    # INSERT 쿼리: product_id만 삽입하고 is_crawled는 기본값(FALSE)을 사용
    # IGNORE 옵션을 사용하여 이미 존재하는 PRIMARY KEY(product_id)는 무시하고 넘어갑니다.
    sql = "INSERT IGNORE INTO crawl_target_id (product_id) VALUES (%s)"
    
    # executemany에 필요한 데이터 형식: [(id1,), (id2,), (id3,), ...]
    # 튜플의 리스트 형태로 변환합니다.
    data_to_insert = [(pid,) for pid in product_id_list]

    try:
        conn = pymysql.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            db=DATABASE,
            port=PORT,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.Cursor
        )
        cursor = conn.cursor()

        print(f"\n--- crawl_target_id에 ID 삽입 시작 ({len(data_to_insert)}개) ---")
        
        # executemany를 사용하여 대량 삽입 실행
        affected_rows = cursor.executemany(sql, data_to_insert)
        conn.commit()
        
        print(f"🎉 crawl_target_id 삽입 완료: {affected_rows}개 행이 새로 추가/갱신되었습니다.")
        
    except pymysql.MySQLError as e:
        print(f"❌ ID 삽입 중 데이터베이스 오류 발생: {e}")
        if conn:
            conn.rollback()
        sys.exit(1)
        
    finally:
        if conn:
            conn.close()
            print("🔗 데이터베이스 연결 해제 완료.")


if __name__ == "__main__":
    
    # 2. ID 삽입 테스트 (예시 데이터)
    insert_target_ids(id_list)
    
    print("\n--- 실행 안내 ---")
    print("create_musinsa_tables() 함수를 주석 해제하여 테이블을 생성할 수 있습니다.")
    print("insert_target_ids(sample_ids) 함수를 주석 해제하여 ID 삽입을 테스트할 수 있습니다.")


--- crawl_target_id에 ID 삽입 시작 (49992개) ---
🎉 crawl_target_id 삽입 완료: 49992개 행이 새로 추가/갱신되었습니다.
🔗 데이터베이스 연결 해제 완료.

--- 실행 안내 ---
create_musinsa_tables() 함수를 주석 해제하여 테이블을 생성할 수 있습니다.
insert_target_ids(sample_ids) 함수를 주석 해제하여 ID 삽입을 테스트할 수 있습니다.
